#Importing dataset and data Preprocessing

In [ ]:
from google.colab import drive 
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive/machine-learning/projects/cleveland

data


In [ ]:
%cd drive/My\ Drive/machine-learning/projects/cleveland

/content/drive/My Drive/machine-learning/projects/cleveland


In [ ]:
import numpy as np
import pandas as pd
import random as rd
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import svm

In [ ]:
Data = pd.read_csv('./data/heart.csv')

In [ ]:
Data.shape

(303, 14)

In [ ]:
Data.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [ ]:
Data = Data.sample(frac=1)

Xold = Data.drop(["target"],axis=1)
Y = pd.DataFrame(Data,columns=["target"]).values

In [ ]:
Xold.shape

(303, 13)

In [ ]:
Xold.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
179,57,1,0,150,276,0,0,112,1,0.6,1,1,1
30,41,0,1,105,198,0,1,168,0,0.0,2,1,2
249,69,1,2,140,254,0,0,146,0,2.0,1,3,3
102,63,0,1,140,195,0,1,179,0,0.0,2,2,2
289,55,0,0,128,205,0,2,130,1,2.0,1,1,3


In [ ]:
Xold.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000


In [ ]:
Xold.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
dtype: int64

In [ ]:
norm = preprocessing.MinMaxScaler()
X = norm.fit_transform(Xold)
X

array([[0.58333333, 1.        , 0.        , ..., 0.5       , 0.25      ,
        0.33333333],
       [0.25      , 0.        , 0.33333333, ..., 1.        , 0.25      ,
        0.66666667],
       [0.83333333, 1.        , 0.66666667, ..., 0.5       , 0.75      ,
        1.        ],
       ...,
       [0.33333333, 1.        , 0.        , ..., 1.        , 0.        ,
        0.66666667],
       [0.75      , 0.        , 0.66666667, ..., 1.        , 0.        ,
        0.66666667],
       [0.60416667, 0.        , 0.66666667, ..., 1.        , 0.        ,
        0.66666667]])

# SVM without GA

In [ ]:
import time
start_time = time.time()
kfold = 10
SVMClass = svm.SVC()
Aa1 = 0

kf = KFold(n_splits= kfold)

for train_index, test_index in kf.split(X):
  X_train,X_test = X[train_index],X[test_index]
  Y_train,Y_test = Y[train_index],Y[test_index]

  model1 = SVMClass
  model1.fit(X_train,Y_train.ravel())
  Pa_1 = model1.predict(X_test)
  AC1 = model1.score(X_test, Y_test)
  
  Aa1 += AC1

print()
print("Accuracy for SVM without GA: %f" %(Aa1/kfold))
print("--- %s seconds ---" % (time.time() - start_time))


Accuracy for SVM without GA: 0.821720
--- 0.03829455375671387 seconds ---


# Hyperparameter tuning using GA on whole dataset and accuracy

In [ ]:
start_time = time.time()
p_c = 1 # prob. of crossover
p_m = 0.2 # prob. of mutation
pop = 80
gen = 50
kfold = 3


#first 15 represent C and the next 15 represent gamma
XY0 = np.array([1,1,1,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,1,1,0,1,0,0,0,1,0,0,1,0])


n_list = np.empty((0,len(XY0)))


for i in range(pop):
    rd.shuffle(XY0)
    n_list = np.vstack((n_list,XY0))


# Calculate the FV

# X = C
a_X = 10
b_X = 1000
l_X = (len(XY0)/2)


# Y = gamma
a_Y = 0.05
b_Y = 0.99
l_Y = (len(XY0)/2)


Precision_X = (b_X - a_X)/((2**l_X)-1)
Precision_Y = (b_Y - a_Y)/((2**l_Y)-1)


z = 0
t = 1
X0_num_Sum = 0

for i in range(len(XY0)//2):
    X0_num = XY0[-t]*2**z
    X0_num_Sum += X0_num
    t = t+1
    z = z+1

p = 0
u = 1+(len(XY0)//2)
Y0_num_Sum = 0

for i in range(len(XY0)//2):
    Y0_num = XY0[-u]*2**p
    Y0_num_Sum += Y0_num
    u = u+1
    p = p+1


Decoded_X = (X0_num_Sum*Precision_X)+a_X
Decoded_Y = (Y0_num_Sum*Precision_Y)+a_Y

print()
print("Decoded_X:",Decoded_X)
print("Decoded_Y:",Decoded_Y)


Final_Best_in_Generation_X = []
Worst_Best_in_Generation_X = []

One_Final_Guy = np.empty((0,len(XY0)+2))
One_Final_Guy_Final = []

Min_for_all_Generations_for_Mut_1 = np.empty((0,len(XY0)+1))
Min_for_all_Generations_for_Mut_2 = np.empty((0,len(XY0)+1))

Min_for_all_Generations_for_Mut_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2 = np.empty((0,len(XY0)+2))

Min_for_all_Generations_for_Mut_1_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2_2 = np.empty((0,len(XY0)+2))


Generation = 1


for i in range(gen):


    New_Population = np.empty((0,len(XY0)))

    All_in_Generation_X_1 = np.empty((0,len(XY0)+1))
    All_in_Generation_X_2 = np.empty((0,len(XY0)+1))

    Min_in_Generation_X_1= []
    Min_in_Generation_X_2 = []


    Save_Best_in_Generation_X = np.empty((0,len(XY0)+1))
    Final_Best_in_Generation_X = []
    Worst_Best_in_Generation_X = []


    print()
    print("GENERATION: #:",Generation)

    Family = 1

    for j in range(int(pop/2)):

        print()
        print("Family: #:",Family)

        # Tour. Selection

        Parents = np.empty((0,len(XY0)))

        for i in range(2):

            Battle_Troops = []

            Warrior_1_index = np.random.randint(0,len(n_list))
            Warrior_2_index = np.random.randint(0,len(n_list))
            Warrior_3_index = np.random.randint(0,len(n_list))

            while Warrior_1_index==Warrior_2_index:
                Warrior_1_index = np.random.randint(0,len(n_list))
            while Warrior_2_index==Warrior_3_index:
                Warrior_3_index = np.random.randint(0,len(n_list))
            while Warrior_1_index==Warrior_3_index:
                Warrior_3_index = np.random.randint(0,len(n_list))



            Warrior_1 = n_list[Warrior_1_index]
            Warrior_2 = n_list[Warrior_2_index]
            Warrior_3 = n_list[Warrior_3_index]

            Battle_Troops = [Warrior_1,Warrior_2,Warrior_3]


            # Warrior 1

            z = 0
            t = 1
            X0_num_Sum_W1 = 0

            for i in range(len(XY0)//2):
                X0_num_W1 = Warrior_1[-t]*2**z
                X0_num_Sum_W1 += X0_num_W1
                t = t+1
                z = z+1

            Y0_num_Sum_W1 = 0
            p = 0
            u = 1+(len(XY0)//2)
            Y0_num_Sum = 0

            for i in range(len(XY0)//2):
                Y0_num_W1 = Warrior_1[-u]*2**p
                Y0_num_Sum_W1 += Y0_num_W1
                u = u+1
                p = p+1


            Decoded_X_W1 = (X0_num_Sum_W1*Precision_X)+a_X
            Decoded_Y_W1 = (Y0_num_Sum_W1*Precision_Y)+a_Y
            '''
            print()
            print("Decoded_X_W1:",Decoded_X_W1)
            print("Decoded_Y_W1:",Decoded_Y_W1)
            '''
            P_1 = 0

            kf = KFold(n_splits=kfold)

            for train_index,test_index in kf.split(X):
                X_train,X_test = X[train_index],X[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVC(kernel="rbf",C=Decoded_X_W1,gamma=Decoded_Y_W1)
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test)

                OF_So_Far_1 = 1-(AC1)

                P_1 += OF_So_Far_1

            OF_So_Far_W1 = P_1/kfold



            # Warrior 2

            z = 0
            t = 1
            X0_num_Sum_W2 = 0

            for i in range(len(XY0)//2):
                X0_num_W1 = Warrior_1[-t]*2**z
                X0_num_Sum_W1 += X0_num_W1
                t = t+1
                z = z+1

            Y0_num_Sum_W2 = 0
            p = 0
            u = 1+(len(XY0)//2)
            Y0_num_Sum = 0

            for i in range(len(XY0)//2):
                Y0_num_W2 = Warrior_2[-u]*2**p
                Y0_num_Sum_W2 += Y0_num_W2
                u = u+1
                p = p+1


            Decoded_X_W2 = (X0_num_Sum_W2*Precision_X)+a_X
            Decoded_Y_W2 = (Y0_num_Sum_W2*Precision_Y)+a_Y
            '''
            print()
            print("Decoded_X_W2:",Decoded_X_W2)
            print("Decoded_Y_W2:",Decoded_Y_W2)
            '''
            P_1 = 0

            kf = KFold(n_splits=kfold)

            for train_index,test_index in kf.split(X):
                X_train,X_test = X[train_index],X[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVC(kernel="rbf",C=Decoded_X_W2,gamma=Decoded_Y_W2)
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test)

                OF_So_Far_2 = 1-(AC1)

                P_1 += OF_So_Far_2

            OF_So_Far_W2 = P_1/kfold

            # Warrior 3

            z = 0
            t = 1
            X0_num_Sum_W3 = 0

            for i in range(len(XY0)//2):
                X0_num_W1 = Warrior_3[-t]*2**z
                X0_num_Sum_W3 += X0_num_W1
                t = t+1
                z = z+1

            Y0_num_Sum_W3 = 0
            p = 0
            u = 1+(len(XY0)//2)
            Y0_num_Sum = 0

            for i in range(len(XY0)//2):
                Y0_num_W3 = Warrior_3[-u]*2**p
                Y0_num_Sum_W3 += Y0_num_W1
                u = u+1
                p = p+1


            Decoded_X_W3 = (X0_num_Sum_W1*Precision_X)+a_X
            Decoded_Y_W3 = (Y0_num_Sum_W1*Precision_Y)+a_Y
            '''
            print()
            print("Decoded_X_W3:",Decoded_X_W3)
            print("Decoded_Y_W3:",Decoded_Y_W3)
            '''
            P_1 = 0

            kf = KFold(n_splits=kfold)

            for train_index,test_index in kf.split(X):
                X_train,X_test = X[train_index],X[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVC(kernel="rbf",C=Decoded_X_W3,gamma=Decoded_Y_W3)
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test)

                OF_So_Far_1 = 1-(AC1)

                P_1 += OF_So_Far_1

            OF_So_Far_W3 = P_1/kfold


            Prize_Warrior_1 = OF_So_Far_W1
            Prize_Warrior_2 = OF_So_Far_W2
            Prize_Warrior_3 = OF_So_Far_W3


            if Prize_Warrior_1 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_1
                Prize = Prize_Warrior_1

            if Prize_Warrior_2 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_2
                Prize = Prize_Warrior_2

            if Prize_Warrior_3 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_3
                Prize = Prize_Warrior_3


            Parents = np.vstack((Parents,Winner))
        '''
        print(Parents)
        '''
        Parent_1 = Parents[0]
        Parent_2 = Parents[1]


        # Crossover

        Child_1 = np.empty((0,len(XY0)))
        Child_2 = np.empty((0,len(XY0)))




        Ran_CO_1 = np.random.rand()

        if Ran_CO_1 < p_c:


            Cr_1 = np.random.randint(0,len(XY0))
            Cr_2 = np.random.randint(0,len(XY0))

            while Cr_1 == Cr_2:
                Cr_2 = np.random.randint(0,len(XY0))

            if Cr_1 < Cr_2:

                Med_Seg_1 = Parent_1[Cr_1:Cr_2+1]
                Med_Seg_2 = Parent_2[Cr_1:Cr_2+1]

                First_Seg_1 = Parent_1[:Cr_1]
                Sec_Seg_1 = Parent_1[Cr_2+1:]

                First_Seg_2 = Parent_2[:Cr_1]
                Sec_Seg_2 = Parent_2[Cr_2+1:]

                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))

            else:

                Med_Seg_1 = Parent_1[Cr_2:Cr_1+1]
                Med_Seg_2 = Parent_2[Cr_2:Cr_1+1]

                First_Seg_1 = Parent_1[:Cr_2]
                Sec_Seg_1 = Parent_1[Cr_1+1:]

                First_Seg_2 = Parent_2[:Cr_2]
                Sec_Seg_2 = Parent_2[Cr_1+1:]

                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))

        else:
            Child_1 = Parent_1
            Child_2 = Parent_2

        '''
        print()
        print("Child_1:",Child_1)
        print("Child_2:",Child_2)
        '''

        Mutated_Child_1 = []

        t = 0

        for i in Child_1:

            Ran_Mut_1 = np.random.rand()

            if Ran_Mut_1 < p_m:

                if Child_1[t] == 0:
                    Child_1[t] = 1
                else:
                    Child_1[t] = 0
                t = t+1

                Mutated_Child_1 = Child_1

            else:
                Mutated_Child_1 = Child_1


        Mutated_Child_2 = []

        t = 0

        for i in Child_2:

            Ran_Mut_2 = np.random.rand()

            if Ran_Mut_2 < p_m:

                if Child_2[t] == 0:
                    Child_2[t] = 1
                else:
                    Child_2[t] = 0
                t = t+1

                Mutated_Child_2 = Child_2

            else:
                Mutated_Child_2 = Child_2
        '''
        print()
        print("Mutated_Child_1:",Mutated_Child_1)
        print("Mutated_Child_2:",Mutated_Child_2)
        '''

        # For mutated child_1

        X0_num_Sum_MC_1 = 0
        Y0_num_Sum_MC_1 = 0

        z = 0
        t = 1


        for i in range(len(XY0)//2):
            X0_num_M1 = Mutated_Child_1[-t]*2**z
            X0_num_Sum_MC_1 += X0_num_M1
            t = t+1
            z = z+1

        p = 0
        u = 1+(len(XY0)//2)

        for i in range(len(XY0)//2):
            Y0_num_M1 = Mutated_Child_1[-u]*2**p
            Y0_num_Sum_MC_1 += Y0_num_M1
            u = u+1
            p = p+1


        Decoded_X_M1 = (X0_num_Sum_MC_1*Precision_X)+a_X
        Decoded_Y_M1 = (Y0_num_Sum_MC_1*Precision_Y)+a_Y
        '''
        print()
        print("Decoded_X_M1:",Decoded_X_M1)
        print("Decoded_Y_M1:",Decoded_Y_M1)
        '''
        P_1 = 0

        kf = KFold(n_splits=kfold)

        for train_index,test_index in kf.split(X):
            X_train,X_test = X[train_index],X[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]

            model1 = svm.SVC(kernel="rbf",C=Decoded_X_M1,gamma=Decoded_Y_M1)
            model1.fit(X_train,Y_train.ravel())
            PL1 = model1.predict(X_test)

            AC1 = model1.score(X_test,Y_test)

            OF_So_Far_1 = 1-(AC1)

            P_1 += OF_So_Far_1

        OF_So_Far_M1 = P_1/kfold


        # For mutated child_2

        X0_num_Sum_MC_2 = 0
        Y0_num_Sum_MC_2 = 0

        z = 0
        t = 1


        for i in range(len(XY0)//2):
            X0_num_M2 = Mutated_Child_2[-t]*2**z
            X0_num_Sum_MC_2 += X0_num_M2
            t = t+1
            z = z+1

        p = 0
        u = 1+(len(XY0)//2)

        for i in range(len(XY0)//2):
            Y0_num_M2 = Mutated_Child_2[-u]*2**p
            Y0_num_Sum_MC_2 += Y0_num_M2
            u = u+1
            p = p+1


        Decoded_X_M2 = (X0_num_Sum_MC_2*Precision_X)+a_X
        Decoded_Y_M2 = (Y0_num_Sum_MC_2*Precision_Y)+a_Y
        '''
        print()
        print("Decoded_X_M2:",Decoded_X_M2)
        print("Decoded_Y_M2:",Decoded_Y_M2)
        '''
        P_1 = 0

        kf = KFold(n_splits=kfold)

        for train_index,test_index in kf.split(X):
            X_train,X_test = X[train_index],X[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]

            model1 = svm.SVC(kernel="rbf",C=Decoded_X_M2,gamma=Decoded_Y_M2)
            model1.fit(X_train,Y_train.ravel())
            PL1 = model1.predict(X_test)

            AC1 = model1.score(X_test,Y_test)

            OF_So_Far_2 = 1-(AC1)

            P_1 += OF_So_Far_2

        OF_So_Far_M2 = P_1/kfold


        print()
        print("FV for Mutated Child #1 at Gen",Generation,":",OF_So_Far_M1)
        print("FV for Mutated Child #2 at Gen",Generation,":",OF_So_Far_M2)


        All_in_Generation_X_1_1_temp = Mutated_Child_1[np.newaxis]

        All_in_Generation_X_1_1 = np.column_stack((OF_So_Far_M1,All_in_Generation_X_1_1_temp))

        All_in_Generation_X_2_2_temp = Mutated_Child_2[np.newaxis]

        All_in_Generation_X_2_2 = np.column_stack((OF_So_Far_M2,All_in_Generation_X_2_2_temp))


        All_in_Generation_X_1 = np.vstack((All_in_Generation_X_1,All_in_Generation_X_1_1))
        All_in_Generation_X_2 = np.vstack((All_in_Generation_X_2,All_in_Generation_X_2_2))


        Save_Best_in_Generation_X = np.vstack((All_in_Generation_X_1,All_in_Generation_X_2))


        New_Population = np.vstack((New_Population,Mutated_Child_1,Mutated_Child_2))

        R_1 = []
        t = 0
        for i in All_in_Generation_X_1:

            if(All_in_Generation_X_1[t,:1]) <= min(All_in_Generation_X_1[:,:1]):
                R_1 = All_in_Generation_X_1[t,:]
            t = t+1


        Min_in_Generation_X_1 = R_1[np.newaxis]


        R_2 = []
        t = 0
        for i in All_in_Generation_X_2:

            if(All_in_Generation_X_2[t,:1]) <= min(All_in_Generation_X_2[:,:1]):
                R_2 = All_in_Generation_X_2[t,:]
            t = t+1


        Min_in_Generation_X_2 = R_2[np.newaxis]


        Family = Family+1


    t = 0
    R_11 = []
    for i in Save_Best_in_Generation_X:

        if (Save_Best_in_Generation_X[t,:1]) <= min(Save_Best_in_Generation_X[:,:1]):
            R_11 = Save_Best_in_Generation_X[t,:]
        t = t+1

    Final_Best_in_Generation_X = R_11[np.newaxis]


    t = 0
    R_22 = []
    for i in Save_Best_in_Generation_X:

        if (Save_Best_in_Generation_X[t,:1]) >= max(Save_Best_in_Generation_X[:,:1]):
            R_22 = Save_Best_in_Generation_X[t,:1]
        t = t+1

    Worst_Best_in_Generation_X = R_22[np.newaxis]


    Darwin_Guy = Final_Best_in_Generation_X[:]
    Not_So_Darwin_Guy = Worst_Best_in_Generation_X[:]


    Darwin_Guy = Darwin_Guy[0:,1:].tolist()
    Not_So_Darwin_Guy = Not_So_Darwin_Guy[0:,1:].tolist()

    Best_1 = np.where((New_Population==Darwin_Guy))
    Worst_1 = np.where((New_Population==Not_So_Darwin_Guy))


    New_Population[Worst_1] = Darwin_Guy

    n_list = New_Population


    Min_for_all_Generations_for_Mut_1 = np.vstack((Min_for_all_Generations_for_Mut_1,
                                                   Min_in_Generation_X_1))
    Min_for_all_Generations_for_Mut_2 = np.vstack((Min_for_all_Generations_for_Mut_2,
                                                   Min_in_Generation_X_2))


    Min_for_all_Generations_for_Mut_1_1 = np.insert(Min_in_Generation_X_1,0,Generation)
    Min_for_all_Generations_for_Mut_2_2 = np.insert(Min_in_Generation_X_2,0,Generation)


    Min_for_all_Generations_for_Mut_1_1_1 = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                                                       Min_for_all_Generations_for_Mut_1_1))

    Min_for_all_Generations_for_Mut_2_2_2 = np.vstack((Min_for_all_Generations_for_Mut_2_2_2,
                                                       Min_for_all_Generations_for_Mut_2_2))


    Generation = Generation+1



One_Final_Guy = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                           Min_for_all_Generations_for_Mut_2_2_2))


t = 0
Final_Here = []

for i in One_Final_Guy:
    if(One_Final_Guy[t,1]) <= min(One_Final_Guy[:,1]):
        Final_Here = One_Final_Guy[t,:]
    t = t+1

One_Final_Guy_Final = Final_Here[np.newaxis]

print()
print("Min in all Generations:",One_Final_Guy_Final)

print()
print("Final Solution:",One_Final_Guy_Final[:,2:])
print("Highest Accuracy:",(1-One_Final_Guy_Final[:,1]))

XY0_Encoded_After = One_Final_Guy_Final[:,2:][0]


z = 0
t = 1
X0_num_Sum_Encoded_After = 0

for i in range(len(XY0)//2):
    X0_num_Encoded_After = XY0_Encoded_After[-t]*2**z
    X0_num_Sum_Encoded_After += X0_num_Encoded_After
    t = t+1
    z = z+1

p = 0
u = 1+(len(XY0)//2)
Y0_num_Sum_Encoded_After = 0

for i in range(len(XY0)//2):
    Y0_num_Encoded_After = XY0_Encoded_After[-u]*2**p
    Y0_num_Sum_Encoded_After += Y0_num_Encoded_After
    u = u+1
    p = p+1


Decoded_X_Encoded_After = (X0_num_Sum_Encoded_After*Precision_X)+a_X
Decoded_Y_Encoded_After = (Y0_num_Sum_Encoded_After*Precision_Y)+a_Y

print()
print("Decoded_X_Encoded_After:",Decoded_X_Encoded_After)
print("Decoded_Y_Encoded_After:",Decoded_Y_Encoded_After)
print("--- %s seconds ---" % (time.time() - start_time))



Decoded_X: 691.3406781212806
Decoded_Y: 0.23686971648304694

GENERATION: #: 1

Family: #: 1

FV for Mutated Child #1 at Gen 1 : 0.20132013201320131
FV for Mutated Child #2 at Gen 1 : 0.1749174917491749

Family: #: 2

FV for Mutated Child #1 at Gen 1 : 0.2178217821782178
FV for Mutated Child #2 at Gen 1 : 0.2211221122112211

Family: #: 3

FV for Mutated Child #1 at Gen 1 : 0.1815181518151815
FV for Mutated Child #2 at Gen 1 : 0.20132013201320131

Family: #: 4

FV for Mutated Child #1 at Gen 1 : 0.2211221122112211
FV for Mutated Child #2 at Gen 1 : 0.2211221122112211

Family: #: 5

FV for Mutated Child #1 at Gen 1 : 0.20132013201320131
FV for Mutated Child #2 at Gen 1 : 0.16831683168316833

Family: #: 6

FV for Mutated Child #1 at Gen 1 : 0.20462046204620463
FV for Mutated Child #2 at Gen 1 : 0.2244224422442244

Family: #: 7

FV for Mutated Child #1 at Gen 1 : 0.21452145214521454
FV for Mutated Child #2 at Gen 1 : 0.20792079207920788

Family: #: 8

FV for Mutated Child #1 at Gen 1 : 0.2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 2 : 0.2244224422442244
FV for Mutated Child #2 at Gen 2 : 0.2277227722772277

Family: #: 3

FV for Mutated Child #1 at Gen 2 : 0.2244224422442244
FV for Mutated Child #2 at Gen 2 : 0.2178217821782178

Family: #: 4

FV for Mutated Child #1 at Gen 2 : 0.2178217821782178
FV for Mutated Child #2 at Gen 2 : 0.2211221122112211

Family: #: 5

FV for Mutated Child #1 at Gen 2 : 0.1881188118811881
FV for Mutated Child #2 at Gen 2 : 0.198019801980198

Family: #: 6

FV for Mutated Child #1 at Gen 2 : 0.2277227722772277
FV for Mutated Child #2 at Gen 2 : 0.18481848184818483

Family: #: 7

FV for Mutated Child #1 at Gen 2 : 0.21452145214521454
FV for Mutated Child #2 at Gen 2 : 0.2079207920792079

Family: #: 8

FV for Mutated Child #1 at Gen 2 : 0.20792079207920788
FV for Mutated Child #2 at Gen 2 : 0.19471947194719472

Family: #: 9

FV for Mutated Child #1 at Gen 2 : 0.20132013201320131
FV for Mutated Child #2 at Gen 2 : 0.2244224422442244

Family: #: 10

FV for Mut

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 3 : 0.2211221122112211
FV for Mutated Child #2 at Gen 3 : 0.1815181518151815

Family: #: 3

FV for Mutated Child #1 at Gen 3 : 0.2211221122112211
FV for Mutated Child #2 at Gen 3 : 0.2211221122112211

Family: #: 4

FV for Mutated Child #1 at Gen 3 : 0.20462046204620463
FV for Mutated Child #2 at Gen 3 : 0.1848184818481848

Family: #: 5

FV for Mutated Child #1 at Gen 3 : 0.1815181518151815
FV for Mutated Child #2 at Gen 3 : 0.2211221122112211

Family: #: 6

FV for Mutated Child #1 at Gen 3 : 0.2211221122112211
FV for Mutated Child #2 at Gen 3 : 0.1848184818481848

Family: #: 7

FV for Mutated Child #1 at Gen 3 : 0.24092409240924093
FV for Mutated Child #2 at Gen 3 : 0.2211221122112211

Family: #: 8

FV for Mutated Child #1 at Gen 3 : 0.2211221122112211
FV for Mutated Child #2 at Gen 3 : 0.1914191419141914

Family: #: 9

FV for Mutated Child #1 at Gen 3 : 0.20132013201320131
FV for Mutated Child #2 at Gen 3 : 0.2178217821782178

Family: #: 10

FV for Muta

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 49 : 0.1848184818481848
FV for Mutated Child #2 at Gen 49 : 0.17821782178217818

Family: #: 2

FV for Mutated Child #1 at Gen 49 : 0.18151815181518152
FV for Mutated Child #2 at Gen 49 : 0.17821782178217824

Family: #: 3

FV for Mutated Child #1 at Gen 49 : 0.18811881188118815
FV for Mutated Child #2 at Gen 49 : 0.1749174917491749

Family: #: 4

FV for Mutated Child #1 at Gen 49 : 0.1815181518151815
FV for Mutated Child #2 at Gen 49 : 0.19801980198019806

Family: #: 5

FV for Mutated Child #1 at Gen 49 : 0.18151815181518152
FV for Mutated Child #2 at Gen 49 : 0.18481848184818483

Family: #: 6

FV for Mutated Child #1 at Gen 49 : 0.1815181518151815
FV for Mutated Child #2 at Gen 49 : 0.18151815181518152

Family: #: 7

FV for Mutated Child #1 at Gen 49 : 0.18151815181518152
FV for Mutated Child #2 at Gen 49 : 0.17821782178217824

Family: #: 8

FV for Mutated Child #1 at Gen 49 : 0.18481848184818483
FV for Mutated Child #2 at Gen 49 : 0.1815181518151815

Fa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 50 : 0.2211221122112211
FV for Mutated Child #2 at Gen 50 : 0.2211221122112211

Family: #: 4

FV for Mutated Child #1 at Gen 50 : 0.2211221122112211
FV for Mutated Child #2 at Gen 50 : 0.20462046204620463

Family: #: 5

FV for Mutated Child #1 at Gen 50 : 0.20132013201320131
FV for Mutated Child #2 at Gen 50 : 0.2211221122112211

Family: #: 6

FV for Mutated Child #1 at Gen 50 : 0.2211221122112211
FV for Mutated Child #2 at Gen 50 : 0.2211221122112211

Family: #: 7

FV for Mutated Child #1 at Gen 50 : 0.2211221122112211
FV for Mutated Child #2 at Gen 50 : 0.20132013201320131

Family: #: 8

FV for Mutated Child #1 at Gen 50 : 0.2211221122112211
FV for Mutated Child #2 at Gen 50 : 0.2211221122112211

Family: #: 9

FV for Mutated Child #1 at Gen 50 : 0.2244224422442244
FV for Mutated Child #2 at Gen 50 : 0.2211221122112211

Family: #: 10

FV for Mutated Child #1 at Gen 50 : 0.2211221122112211
FV for Mutated Child #2 at Gen 50 : 0.198019801980198

Family: #:

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [ ]:
start_time = time.time()
kfold = 10

SVMClass = svm.SVC()

Aa1 = 0

kf = KFold(n_splits=kfold)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    model1 = SVMClass
    model1.fit(X_train, Y_train.ravel())
    Pa_1=model1.predict(X_test)
    AC1=model1.score(X_test,Y_test)
    
    Aa1 += AC1
       
print()
print("Accuracy for SVM without GA: %f" % (Aa1/kfold))



SVMClass = svm.SVC(kernel='rbf', C=11, gamma=0.05)

Aa1 = 0

kf = KFold(n_splits=kfold)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    model1 = SVMClass
    model1.fit(X_train, Y_train.ravel())
    Pa_1=model1.predict(X_test)
    AC1=model1.score(X_test,Y_test)
    
    Aa1 += AC1
       
print()
print("Accuracy for SVM with GA: %f" % (Aa1/kfold))
print("--- %s seconds ---" % (time.time() - start_time))


Accuracy for SVM without GA: 0.821720

Accuracy for SVM with GA: 0.831505
--- 0.0665895938873291 seconds ---


# Feature selection using GA

In [ ]:
XY0 = np.array([1,1,1,1,0,0,1,0,1,0,0,1,1])

In [ ]:
start_time = time.time()
p_c = 1 # prob. of crossover
p_m = 0.2 # prob. of mutation
pop = 80
gen = 30
kfold = 5

n_list = np.empty((0,len(XY0)))

for i in range(pop):
    rd.shuffle(XY0)
    n_list = np.vstack((n_list,XY0))


Final_Best_in_Generation_X = []
Worst_Best_in_Generation_X = []

One_Final_Guy = np.empty((0,len(XY0)+2))
One_Final_Guy_Final = []

Min_for_all_Generations_for_Mut_1 = np.empty((0,len(XY0)+1))
Min_for_all_Generations_for_Mut_2 = np.empty((0,len(XY0)+1))

Min_for_all_Generations_for_Mut_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2 = np.empty((0,len(XY0)+2))

Min_for_all_Generations_for_Mut_1_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2_2 = np.empty((0,len(XY0)+2))


Generation = 1


for i in range(gen):
    
    
    New_Population = np.empty((0,len(XY0)))
    
    All_in_Generation_X_1 = np.empty((0,len(XY0)+1))
    All_in_Generation_X_2 = np.empty((0,len(XY0)+1))
    
    Min_in_Generation_X_1= []
    Min_in_Generation_X_2 = []
    
    
    Save_Best_in_Generation_X = np.empty((0,len(XY0)+1))
    Final_Best_in_Generation_X = []
    Worst_Best_in_Generation_X = []
    
    
    print()
    print("GENERATION: #:",Generation)
        
    Family = 1
    
    for j in range(int(pop/2)):
        
        print()
        print("Family: #:",Family)
        
        # Tour. Selection
        
        Parents = np.empty((0,len(XY0)))
        
        for i in range(2):
            
            Battle_Troops = []
            
            Warrior_1_index = np.random.randint(0,len(n_list))
            Warrior_2_index = np.random.randint(0,len(n_list))
            Warrior_3_index = np.random.randint(0,len(n_list))
            
            while Warrior_1_index==Warrior_2_index:
                Warrior_1_index = np.random.randint(0,len(n_list))
            while Warrior_2_index==Warrior_3_index:
                Warrior_3_index = np.random.randint(0,len(n_list))
            while Warrior_1_index==Warrior_3_index:
                Warrior_3_index = np.random.randint(0,len(n_list))



            Warrior_1 = n_list[Warrior_1_index]
            Warrior_2 = n_list[Warrior_2_index]
            Warrior_3 = n_list[Warrior_3_index]
            
            Battle_Troops = [Warrior_1,Warrior_2,Warrior_3]
            
            
            # Warrior 1
             
            t = 0

            emp_list = []
            
            for i in Warrior_1:
                if Warrior_1[t] == 1:
                    emp_list.append(t)
                t = t+1
            
            NewX = X[:,emp_list]
            
            P_1 = 0
            
            kf = KFold(n_splits=kfold)
            
            for train_index,test_index in kf.split(X):
                X_train,X_test = NewX[train_index],NewX[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]
                
                model1 = svm.SVC()
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)
                
                AC1 = model1.score(X_test,Y_test)
                
                OF_So_Far_1 = 1-(AC1)
                
                P_1 += OF_So_Far_1
                
            OF_So_Far_W1 = P_1/kfold
            
            
            
            # Warrior 2
            
            t = 0

            emp_list = []
            
            for i in Warrior_2:
                if Warrior_2[t] == 1:
                    emp_list.append(t)
                t = t+1
            
            NewX = X[:,emp_list]
            
            P_2 = 0
            
            kf = KFold(n_splits=kfold)
            
            for train_index,test_index in kf.split(X):
                X_train,X_test = NewX[train_index],NewX[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]
                
                model1 = svm.SVC()
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)
                
                AC1 = model1.score(X_test,Y_test)
                
                OF_So_Far_2 = 1-(AC1)
                
                P_2 += OF_So_Far_2
                
            OF_So_Far_W2 = P_2/kfold
            
            
            
            # Warrior 3
            
            t = 0

            emp_list = []
            
            for i in Warrior_3:
                if Warrior_3[t] == 1:
                    emp_list.append(t)
                t = t+1
            
            NewX = X[:,emp_list]
            
            P_3 = 0
            
            kf = KFold(n_splits=kfold)
            
            for train_index,test_index in kf.split(X):
                X_train,X_test = NewX[train_index],NewX[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]
                
                model1 = svm.SVC()
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)
                
                AC1 = model1.score(X_test,Y_test)
                
                OF_So_Far_3 = 1-(AC1)
                
                P_3 += OF_So_Far_3
                
            OF_So_Far_W3 = P_3/kfold

            
            Prize_Warrior_1 = OF_So_Far_W1
            Prize_Warrior_2 = OF_So_Far_W2
            Prize_Warrior_3 = OF_So_Far_W3
            
            
            if Prize_Warrior_1 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_1
                Prize = Prize_Warrior_1

            if Prize_Warrior_2 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_2
                Prize = Prize_Warrior_2
            
            if Prize_Warrior_3 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_3
                Prize = Prize_Warrior_3


            Parents = np.vstack((Parents,Winner))
        '''
        print(Parents)
        '''
        Parent_1 = Parents[0]
        Parent_2 = Parents[1]
        
        
        # Crossover
        
        Child_1 = np.empty((0,len(XY0)))
        Child_2 = np.empty((0,len(XY0)))
        
        
        
        
        Ran_CO_1 = np.random.rand()
        
        if Ran_CO_1 < p_c:
            
            
            Cr_1 = np.random.randint(0,len(XY0))
            Cr_2 = np.random.randint(0,len(XY0))
            
            while Cr_1 == Cr_2:
                Cr_2 = np.random.randint(0,len(XY0))
                
            if Cr_1 < Cr_2:
                
                Med_Seg_1 = Parent_1[Cr_1:Cr_2+1]
                Med_Seg_2 = Parent_2[Cr_1:Cr_2+1]
                
                First_Seg_1 = Parent_1[:Cr_1]
                Sec_Seg_1 = Parent_1[Cr_2+1:]
                
                First_Seg_2 = Parent_2[:Cr_1]
                Sec_Seg_2 = Parent_2[Cr_2+1:]
                
                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))
                
            else:
                
                Med_Seg_1 = Parent_1[Cr_2:Cr_1+1]
                Med_Seg_2 = Parent_2[Cr_2:Cr_1+1]
                
                First_Seg_1 = Parent_1[:Cr_2]
                Sec_Seg_1 = Parent_1[Cr_1+1:]
                
                First_Seg_2 = Parent_2[:Cr_2]
                Sec_Seg_2 = Parent_2[Cr_1+1:]
                
                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))
        
        else:
            Child_1 = Parent_1
            Child_2 = Parent_2
            
        '''
        print()
        print("Child_1:",Child_1)
        print("Child_2:",Child_2)
        '''
        
        Mutated_Child_1 = []
        
        t = 0
        
        for i in Child_1:
            
            Ran_Mut_1 = np.random.rand()
            
            if Ran_Mut_1 < p_m:
                
                if Child_1[t] == 0:
                    Child_1[t] = 1
                else:
                    Child_1[t] = 0
                t = t+1
                
                Mutated_Child_1 = Child_1
                
            else:
                Mutated_Child_1 = Child_1
                
        
        Mutated_Child_2 = []
        
        t = 0
        
        for i in Child_2:
            
            Ran_Mut_2 = np.random.rand()
            
            if Ran_Mut_2 < p_m:
                
                if Child_2[t] == 0:
                    Child_2[t] = 1
                else:
                    Child_2[t] = 0
                t = t+1
                
                Mutated_Child_2 = Child_2
                
            else:
                Mutated_Child_2 = Child_2
        '''
        print()
        print("Mutated_Child_1:",Mutated_Child_1)
        print("Mutated_Child_2:",Mutated_Child_2)
        '''
        
        # For mutated child_1
        
        t = 0

        emp_list = []
        
        for i in Mutated_Child_1:
            if Mutated_Child_1[t] == 1:
                emp_list.append(t)
            t = t+1
        
        NewX = X[:,emp_list]
        
        P_1 = 0
        
        kf = KFold(n_splits=kfold)
        
        for train_index,test_index in kf.split(X):
            X_train,X_test = NewX[train_index],NewX[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]
            
            model1 = svm.SVC()
            model1.fit(X_train,Y_train.ravel())
            PL1 = model1.predict(X_test)
            
            AC1 = model1.score(X_test,Y_test)
            
            OF_So_Far_1 = 1-(AC1)
            
            P_1 += OF_So_Far_1
            
        OF_So_Far_M1 = P_1/kfold


        # For mutated child_2
        
        t = 0

        emp_list = []
        
        for i in Mutated_Child_2:
            if Mutated_Child_2[t] == 1:
                emp_list.append(t)
            t = t+1
        
        NewX = X[:,emp_list]
        
        P_2 = 0
        
        kf = KFold(n_splits=kfold)
        
        for train_index,test_index in kf.split(X):
            X_train,X_test = NewX[train_index],NewX[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]
            
            model1 = svm.SVC()
            model1.fit(X_train,Y_train.ravel())
            PL1 = model1.predict(X_test)
            
            AC1 = model1.score(X_test,Y_test)
            
            OF_So_Far_2 = 1-(AC1)
            
            P_2 += OF_So_Far_2
            
        OF_So_Far_M2 = P_2/kfold
        
        
        print()
        print("FV for Mutated Child #1 at Gen",Generation,":",OF_So_Far_M1)
        print("FV for Mutated Child #2 at Gen",Generation,":",OF_So_Far_M2)


        All_in_Generation_X_1_1_temp = Mutated_Child_1[np.newaxis]
        
        All_in_Generation_X_1_1 = np.column_stack((OF_So_Far_M1,All_in_Generation_X_1_1_temp))
        
        All_in_Generation_X_2_2_temp = Mutated_Child_2[np.newaxis]
        
        All_in_Generation_X_2_2 = np.column_stack((OF_So_Far_M2,All_in_Generation_X_2_2_temp))
        
        
        All_in_Generation_X_1 = np.vstack((All_in_Generation_X_1,All_in_Generation_X_1_1))
        All_in_Generation_X_2 = np.vstack((All_in_Generation_X_2,All_in_Generation_X_2_2))
        
        
        Save_Best_in_Generation_X = np.vstack((All_in_Generation_X_1,All_in_Generation_X_2))


        New_Population = np.vstack((New_Population,Mutated_Child_1,Mutated_Child_2))
        
        R_1 = []
        t = 0
        for i in All_in_Generation_X_1:
            
            if(All_in_Generation_X_1[t,:1]) <= min(All_in_Generation_X_1[:,:1]):
                R_1 = All_in_Generation_X_1[t,:]
            t = t+1
            
        
        Min_in_Generation_X_1 = R_1[np.newaxis]
        
        
        R_2 = []
        t = 0
        for i in All_in_Generation_X_2:
            
            if(All_in_Generation_X_2[t,:1]) <= min(All_in_Generation_X_2[:,:1]):
                R_2 = All_in_Generation_X_2[t,:]
            t = t+1
            
        
        Min_in_Generation_X_2 = R_2[np.newaxis]
        
        
        Family = Family+1
        
        
    t = 0
    R_11 = []
    for i in Save_Best_in_Generation_X:
        
        if (Save_Best_in_Generation_X[t,:1]) <= min(Save_Best_in_Generation_X[:,:1]):
            R_11 = Save_Best_in_Generation_X[t,:]
        t = t+1
            
    Final_Best_in_Generation_X = R_11[np.newaxis]
    
    
    t = 0
    R_22 = []
    for i in Save_Best_in_Generation_X:
        
        if (Save_Best_in_Generation_X[t,:1]) >= max(Save_Best_in_Generation_X[:,:1]):
            R_22 = Save_Best_in_Generation_X[t,:1]
        t = t+1
            
    Worst_Best_in_Generation_X = R_22[np.newaxis]
    
    
    Darwin_Guy = Final_Best_in_Generation_X[:]
    Not_So_Darwin_Guy = Worst_Best_in_Generation_X[:]
    
    
    Darwin_Guy = Darwin_Guy[0:,1:].tolist()
    Not_So_Darwin_Guy = Not_So_Darwin_Guy[0:,1:].tolist()
    
    Best_1 = np.where((New_Population==Darwin_Guy))
    Worst_1 = np.where((New_Population==Not_So_Darwin_Guy))
    
    
    New_Population[Worst_1] = Darwin_Guy
    
    n_list = New_Population

    
    Min_for_all_Generations_for_Mut_1 = np.vstack((Min_for_all_Generations_for_Mut_1,
                                                   Min_in_Generation_X_1))
    Min_for_all_Generations_for_Mut_2 = np.vstack((Min_for_all_Generations_for_Mut_2,
                                                   Min_in_Generation_X_2))
    
    
    Min_for_all_Generations_for_Mut_1_1 = np.insert(Min_in_Generation_X_1,0,Generation)
    Min_for_all_Generations_for_Mut_2_2 = np.insert(Min_in_Generation_X_2,0,Generation)
    
    
    Min_for_all_Generations_for_Mut_1_1_1 = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                                                       Min_for_all_Generations_for_Mut_1_1))
    
    Min_for_all_Generations_for_Mut_2_2_2 = np.vstack((Min_for_all_Generations_for_Mut_2_2_2,
                                                       Min_for_all_Generations_for_Mut_2_2))
    
    
    Generation = Generation+1
    
    
    
One_Final_Guy = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                           Min_for_all_Generations_for_Mut_2_2_2))


t = 0
Final_Here = []

for i in One_Final_Guy:
    if(One_Final_Guy[t,1]) <= min(One_Final_Guy[:,1]):
        Final_Here = One_Final_Guy[t,:]
    t = t+1

One_Final_Guy_Final = Final_Here[np.newaxis]

print()
print("Min in all Generations:",One_Final_Guy_Final)

print()
print("Final Solution:",One_Final_Guy_Final[:,2:])
print("Highest Accuracy:",(1-One_Final_Guy_Final[:,1]))



A11 = One_Final_Guy_Final[:,2:][0]

t = 0

emp_list = []

for i in A11:
    if A11[t] == 1:
        emp_list.append(t)
    t = t+1

print()
print("Features included are:",emp_list)
print("--- %s seconds ---" % (time.time() - start_time))


GENERATION: #: 1

Family: #: 1

FV for Mutated Child #1 at Gen 1 : 0.23792349726775958
FV for Mutated Child #2 at Gen 1 : 0.2014754098360656

Family: #: 2

FV for Mutated Child #1 at Gen 1 : 0.17841530054644808
FV for Mutated Child #2 at Gen 1 : 0.21464480874316944

Family: #: 3

FV for Mutated Child #1 at Gen 1 : 0.20136612021857925
FV for Mutated Child #2 at Gen 1 : 0.21480874316939894

Family: #: 4

FV for Mutated Child #1 at Gen 1 : 0.2081420765027322
FV for Mutated Child #2 at Gen 1 : 0.22442622950819674

Family: #: 5

FV for Mutated Child #1 at Gen 1 : 0.23131147540983604
FV for Mutated Child #2 at Gen 1 : 0.23431693989071034

Family: #: 6

FV for Mutated Child #1 at Gen 1 : 0.19836065573770492
FV for Mutated Child #2 at Gen 1 : 0.17513661202185793

Family: #: 7

FV for Mutated Child #1 at Gen 1 : 0.2508196721311476
FV for Mutated Child #2 at Gen 1 : 0.29743169398907104

Family: #: 8

FV for Mutated Child #1 at Gen 1 : 0.22136612021857927
FV for Mutated Child #2 at Gen 1 : 0.178

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:472: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 2 : 0.16846994535519128
FV for Mutated Child #2 at Gen 2 : 0.1816939890710383

Family: #: 3

FV for Mutated Child #1 at Gen 2 : 0.20775956284153008
FV for Mutated Child #2 at Gen 2 : 0.1817486338797814

Family: #: 4

FV for Mutated Child #1 at Gen 2 : 0.16524590163934427
FV for Mutated Child #2 at Gen 2 : 0.23109289617486342

Family: #: 5

FV for Mutated Child #1 at Gen 2 : 0.1882513661202186
FV for Mutated Child #2 at Gen 2 : 0.23120218579234972

Family: #: 6

FV for Mutated Child #1 at Gen 2 : 0.15885245901639342
FV for Mutated Child #2 at Gen 2 : 0.22114754098360656

Family: #: 7

FV for Mutated Child #1 at Gen 2 : 0.20797814207650273
FV for Mutated Child #2 at Gen 2 : 0.2079781420765027

Family: #: 8

FV for Mutated Child #1 at Gen 2 : 0.22114754098360656
FV for Mutated Child #2 at Gen 2 : 0.20797814207650273

Family: #: 9

FV for Mutated Child #1 at Gen 2 : 0.2080327868852459
FV for Mutated Child #2 at Gen 2 : 0.21459016393442623

Family: #: 10

FV 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:472: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 3 : 0.19830601092896177
FV for Mutated Child #2 at Gen 3 : 0.22114754098360656

Family: #: 3

FV for Mutated Child #1 at Gen 3 : 0.16857923497267763
FV for Mutated Child #2 at Gen 3 : 0.20797814207650273

Family: #: 4

FV for Mutated Child #1 at Gen 3 : 0.15551912568306012
FV for Mutated Child #2 at Gen 3 : 0.16191256830601095

Family: #: 5

FV for Mutated Child #1 at Gen 3 : 0.16202185792349727
FV for Mutated Child #2 at Gen 3 : 0.23120218579234972

Family: #: 6

FV for Mutated Child #1 at Gen 3 : 0.22131147540983606
FV for Mutated Child #2 at Gen 3 : 0.23781420765027322

Family: #: 7

FV for Mutated Child #1 at Gen 3 : 0.19491803278688521
FV for Mutated Child #2 at Gen 3 : 0.23797814207650275

Family: #: 8

FV for Mutated Child #1 at Gen 3 : 0.17513661202185793
FV for Mutated Child #2 at Gen 3 : 0.20792349726775958

Family: #: 9

FV for Mutated Child #1 at Gen 3 : 0.16519125683060112
FV for Mutated Child #2 at Gen 3 : 0.2080327868852459

Family: #: 10


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:472: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 29 : 0.16191256830601095
FV for Mutated Child #2 at Gen 29 : 0.1586338797814208

Family: #: 3

FV for Mutated Child #1 at Gen 29 : 0.1980327868852459
FV for Mutated Child #2 at Gen 29 : 0.1980327868852459

Family: #: 4

FV for Mutated Child #1 at Gen 29 : 0.2113114754098361
FV for Mutated Child #2 at Gen 29 : 0.1948087431693989

Family: #: 5

FV for Mutated Child #1 at Gen 29 : 0.1586338797814208
FV for Mutated Child #2 at Gen 29 : 0.21469945355191258

Family: #: 6

FV for Mutated Child #1 at Gen 29 : 0.1553005464480874
FV for Mutated Child #2 at Gen 29 : 0.15852459016393444

Family: #: 7

FV for Mutated Child #1 at Gen 29 : 0.16513661202185792
FV for Mutated Child #2 at Gen 29 : 0.20136612021857925

Family: #: 8

FV for Mutated Child #1 at Gen 29 : 0.15863387978142077
FV for Mutated Child #2 at Gen 29 : 0.19469945355191257

Family: #: 9

FV for Mutated Child #1 at Gen 29 : 0.1980327868852459
FV for Mutated Child #2 at Gen 29 : 0.19480874316939892

Famil

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:472: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 30 : 0.20797814207650273
FV for Mutated Child #2 at Gen 30 : 0.1980327868852459

Family: #: 3

FV for Mutated Child #1 at Gen 30 : 0.16508196721311474
FV for Mutated Child #2 at Gen 30 : 0.2080327868852459

Family: #: 4

FV for Mutated Child #1 at Gen 30 : 0.16513661202185792
FV for Mutated Child #2 at Gen 30 : 0.16508196721311474

Family: #: 5

FV for Mutated Child #1 at Gen 30 : 0.2079781420765027
FV for Mutated Child #2 at Gen 30 : 0.21459016393442623

Family: #: 6

FV for Mutated Child #1 at Gen 30 : 0.1586338797814208
FV for Mutated Child #2 at Gen 30 : 0.15852459016393444

Family: #: 7

FV for Mutated Child #1 at Gen 30 : 0.15852459016393444
FV for Mutated Child #2 at Gen 30 : 0.21464480874316944

Family: #: 8

FV for Mutated Child #1 at Gen 30 : 0.1586338797814208
FV for Mutated Child #2 at Gen 30 : 0.21459016393442623

Family: #: 9

FV for Mutated Child #1 at Gen 30 : 0.1586338797814208
FV for Mutated Child #2 at Gen 30 : 0.20797814207650273

Fam

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:472: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


# Hyperparamenter tuning on selected features

In [ ]:
X = pd.DataFrame(X)
X.columns = Xold.columns
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.583333,1.0,0.000000,0.528302,0.342466,0.0,0.0,0.312977,1.0,0.096774,0.5,0.25,0.333333
1,0.250000,0.0,0.333333,0.103774,0.164384,0.0,0.5,0.740458,0.0,0.000000,1.0,0.25,0.666667
2,0.833333,1.0,0.666667,0.433962,0.292237,0.0,0.0,0.572519,0.0,0.322581,0.5,0.75,1.000000
3,0.708333,0.0,0.333333,0.433962,0.157534,0.0,0.5,0.824427,0.0,0.000000,1.0,0.50,0.666667
4,0.541667,0.0,0.000000,0.320755,0.180365,0.0,1.0,0.450382,1.0,0.322581,0.5,0.25,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.541667,0.0,0.333333,0.386792,0.283105,0.0,0.0,0.687023,0.0,0.225806,0.5,0.00,0.666667
299,0.312500,1.0,0.000000,0.245283,0.098174,0.0,0.5,0.557252,1.0,0.451613,0.0,0.00,0.333333
300,0.333333,1.0,0.000000,0.198113,0.305936,0.0,0.0,0.870229,0.0,0.000000,1.0,0.00,0.666667
301,0.750000,0.0,0.666667,0.575472,0.326484,0.0,0.5,0.587786,0.0,0.129032,1.0,0.00,0.666667


In [ ]:
X.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
      dtype='object')

In [ ]:
X = X.drop(['sex','chol','exang','slope'],axis=1)

In [ ]:
X.shape

(303, 9)

In [ ]:
X.head()

,age,cp,trestbps,fbs,restecg,thalach,oldpeak,ca,thal
0,0.583333,0.000000,0.528302,0.0,0.0,0.312977,0.096774,0.25,0.333333
1,0.250000,0.333333,0.103774,0.0,0.5,0.740458,0.000000,0.25,0.666667
2,0.833333,0.666667,0.433962,0.0,0.0,0.572519,0.322581,0.75,1.000000
3,0.708333,0.333333,0.433962,0.0,0.5,0.824427,0.000000,0.50,0.666667
4,0.541667,0.000000,0.320755,0.0,1.0,0.450382,0.322581,0.25,1.000000


In [ ]:
start_time = time.time()
p_c = 1 # prob. of crossover
p_m = 0.2 # prob. of mutation
pop = 80
gen = 50
kfold = 3



XY0 = np.array([1,1,1,1,0,0,1,1,0,0,0,1,1,0,0,1,0,1,1,0,1,0,0,1,1,1,0,0,1,0])


n_list = np.empty((0,len(XY0)))


for i in range(pop):
    rd.shuffle(XY0)
    n_list = np.vstack((n_list,XY0))


# Calculate the FV

# X = C
a_X = 10
b_X = 1000
l_X = (len(XY0)/2)


# Y = gamma
a_Y = 0.05
b_Y = 0.99
l_Y = (len(XY0)/2)


Precision_X = (b_X - a_X)/((2**l_X)-1)
Precision_Y = (b_Y - a_Y)/((2**l_Y)-1)


z = 0
t = 1
X0_num_Sum = 0

for i in range(len(XY0)//2):
    X0_num = XY0[-t]*2**z
    X0_num_Sum += X0_num
    t = t+1
    z = z+1

p = 0
u = 1+(len(XY0)//2)
Y0_num_Sum = 0

for i in range(len(XY0)//2):
    Y0_num = XY0[-u]*2**p
    Y0_num_Sum += Y0_num
    u = u+1
    p = p+1


Decoded_X = (X0_num_Sum*Precision_X)+a_X
Decoded_Y = (Y0_num_Sum*Precision_Y)+a_Y

print()
print("Decoded_X:",Decoded_X)
print("Decoded_Y:",Decoded_Y)


Final_Best_in_Generation_X = []
Worst_Best_in_Generation_X = []

One_Final_Guy = np.empty((0,len(XY0)+2))
One_Final_Guy_Final = []

Min_for_all_Generations_for_Mut_1 = np.empty((0,len(XY0)+1))
Min_for_all_Generations_for_Mut_2 = np.empty((0,len(XY0)+1))

Min_for_all_Generations_for_Mut_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2 = np.empty((0,len(XY0)+2))

Min_for_all_Generations_for_Mut_1_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2_2 = np.empty((0,len(XY0)+2))


Generation = 1


for i in range(gen):


    New_Population = np.empty((0,len(XY0)))

    All_in_Generation_X_1 = np.empty((0,len(XY0)+1))
    All_in_Generation_X_2 = np.empty((0,len(XY0)+1))

    Min_in_Generation_X_1= []
    Min_in_Generation_X_2 = []


    Save_Best_in_Generation_X = np.empty((0,len(XY0)+1))
    Final_Best_in_Generation_X = []
    Worst_Best_in_Generation_X = []


    print()
    print("GENERATION: #:",Generation)

    Family = 1

    for j in range(int(pop/2)):

        print()
        print("Family: #:",Family)

        # Tour. Selection

        Parents = np.empty((0,len(XY0)))

        for i in range(2):

            Battle_Troops = []

            Warrior_1_index = np.random.randint(0,len(n_list))
            Warrior_2_index = np.random.randint(0,len(n_list))
            Warrior_3_index = np.random.randint(0,len(n_list))

            while Warrior_1_index==Warrior_2_index:
                Warrior_1_index = np.random.randint(0,len(n_list))
            while Warrior_2_index==Warrior_3_index:
                Warrior_3_index = np.random.randint(0,len(n_list))
            while Warrior_1_index==Warrior_3_index:
                Warrior_3_index = np.random.randint(0,len(n_list))



            Warrior_1 = n_list[Warrior_1_index]
            Warrior_2 = n_list[Warrior_2_index]
            Warrior_3 = n_list[Warrior_3_index]

            Battle_Troops = [Warrior_1,Warrior_2,Warrior_3]


            # Warrior 1

            z = 0
            t = 1
            X0_num_Sum_W1 = 0

            for i in range(len(XY0)//2):
                X0_num_W1 = Warrior_1[-t]*2**z
                X0_num_Sum_W1 += X0_num_W1
                t = t+1
                z = z+1

            Y0_num_Sum_W1 = 0
            p = 0
            u = 1+(len(XY0)//2)
            Y0_num_Sum = 0

            for i in range(len(XY0)//2):
                Y0_num_W1 = Warrior_1[-u]*2**p
                Y0_num_Sum_W1 += Y0_num_W1
                u = u+1
                p = p+1


            Decoded_X_W1 = (X0_num_Sum_W1*Precision_X)+a_X
            Decoded_Y_W1 = (Y0_num_Sum_W1*Precision_Y)+a_Y
            '''
            print()
            print("Decoded_X_W1:",Decoded_X_W1)
            print("Decoded_Y_W1:",Decoded_Y_W1)
            '''
            P_1 = 0

            kf = KFold(n_splits=kfold)

            for train_index,test_index in kf.split(X):
                X_train,X_test = X.iloc[train_index],X.iloc[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVC(kernel="rbf",C=Decoded_X_W1,gamma=Decoded_Y_W1)
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test)

                OF_So_Far_1 = 1-(AC1)

                P_1 += OF_So_Far_1

            OF_So_Far_W1 = P_1/kfold



            # Warrior 2

            z = 0
            t = 1
            X0_num_Sum_W2 = 0

            for i in range(len(XY0)//2):
                X0_num_W1 = Warrior_1[-t]*2**z
                X0_num_Sum_W1 += X0_num_W1
                t = t+1
                z = z+1

            Y0_num_Sum_W2 = 0
            p = 0
            u = 1+(len(XY0)//2)
            Y0_num_Sum = 0

            for i in range(len(XY0)//2):
                Y0_num_W2 = Warrior_2[-u]*2**p
                Y0_num_Sum_W2 += Y0_num_W2
                u = u+1
                p = p+1


            Decoded_X_W2 = (X0_num_Sum_W2*Precision_X)+a_X
            Decoded_Y_W2 = (Y0_num_Sum_W2*Precision_Y)+a_Y
            '''
            print()
            print("Decoded_X_W2:",Decoded_X_W2)
            print("Decoded_Y_W2:",Decoded_Y_W2)
            '''
            P_1 = 0

            kf = KFold(n_splits=kfold)

            for train_index,test_index in kf.split(X):
                X_train,X_test = X.iloc[train_index],X.iloc[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVC(kernel="rbf",C=Decoded_X_W2,gamma=Decoded_Y_W2)
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test)

                OF_So_Far_2 = 1-(AC1)

                P_1 += OF_So_Far_2

            OF_So_Far_W2 = P_1/kfold

            # Warrior 3

            z = 0
            t = 1
            X0_num_Sum_W3 = 0

            for i in range(len(XY0)//2):
                X0_num_W1 = Warrior_3[-t]*2**z
                X0_num_Sum_W3 += X0_num_W1
                t = t+1
                z = z+1

            Y0_num_Sum_W3 = 0
            p = 0
            u = 1+(len(XY0)//2)
            Y0_num_Sum = 0

            for i in range(len(XY0)//2):
                Y0_num_W3 = Warrior_3[-u]*2**p
                Y0_num_Sum_W3 += Y0_num_W1
                u = u+1
                p = p+1


            Decoded_X_W3 = (X0_num_Sum_W1*Precision_X)+a_X
            Decoded_Y_W3 = (Y0_num_Sum_W1*Precision_Y)+a_Y
            '''
            print()
            print("Decoded_X_W3:",Decoded_X_W3)
            print("Decoded_Y_W3:",Decoded_Y_W3)
            '''
            P_1 = 0

            kf = KFold(n_splits=kfold)

            for train_index,test_index in kf.split(X):
                X_train,X_test = X.iloc[train_index],X.iloc[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVC(kernel="rbf",C=Decoded_X_W3,gamma=Decoded_Y_W3)
                model1.fit(X_train,Y_train.ravel())
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test)

                OF_So_Far_1 = 1-(AC1)

                P_1 += OF_So_Far_1

            OF_So_Far_W3 = P_1/kfold


            Prize_Warrior_1 = OF_So_Far_W1
            Prize_Warrior_2 = OF_So_Far_W2
            Prize_Warrior_3 = OF_So_Far_W3


            if Prize_Warrior_1 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_1
                Prize = Prize_Warrior_1

            if Prize_Warrior_2 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_2
                Prize = Prize_Warrior_2

            if Prize_Warrior_3 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_3
                Prize = Prize_Warrior_3


            Parents = np.vstack((Parents,Winner))
        '''
        print(Parents)
        '''
        Parent_1 = Parents[0]
        Parent_2 = Parents[1]


        # Crossover

        Child_1 = np.empty((0,len(XY0)))
        Child_2 = np.empty((0,len(XY0)))




        Ran_CO_1 = np.random.rand()

        if Ran_CO_1 < p_c:


            Cr_1 = np.random.randint(0,len(XY0))
            Cr_2 = np.random.randint(0,len(XY0))

            while Cr_1 == Cr_2:
                Cr_2 = np.random.randint(0,len(XY0))

            if Cr_1 < Cr_2:

                Med_Seg_1 = Parent_1[Cr_1:Cr_2+1]
                Med_Seg_2 = Parent_2[Cr_1:Cr_2+1]

                First_Seg_1 = Parent_1[:Cr_1]
                Sec_Seg_1 = Parent_1[Cr_2+1:]

                First_Seg_2 = Parent_2[:Cr_1]
                Sec_Seg_2 = Parent_2[Cr_2+1:]

                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))

            else:

                Med_Seg_1 = Parent_1[Cr_2:Cr_1+1]
                Med_Seg_2 = Parent_2[Cr_2:Cr_1+1]

                First_Seg_1 = Parent_1[:Cr_2]
                Sec_Seg_1 = Parent_1[Cr_1+1:]

                First_Seg_2 = Parent_2[:Cr_2]
                Sec_Seg_2 = Parent_2[Cr_1+1:]

                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))

        else:
            Child_1 = Parent_1
            Child_2 = Parent_2

        '''
        print()
        print("Child_1:",Child_1)
        print("Child_2:",Child_2)
        '''

        Mutated_Child_1 = []

        t = 0

        for i in Child_1:

            Ran_Mut_1 = np.random.rand()

            if Ran_Mut_1 < p_m:

                if Child_1[t] == 0:
                    Child_1[t] = 1
                else:
                    Child_1[t] = 0
                t = t+1

                Mutated_Child_1 = Child_1

            else:
                Mutated_Child_1 = Child_1


        Mutated_Child_2 = []

        t = 0

        for i in Child_2:

            Ran_Mut_2 = np.random.rand()

            if Ran_Mut_2 < p_m:

                if Child_2[t] == 0:
                    Child_2[t] = 1
                else:
                    Child_2[t] = 0
                t = t+1

                Mutated_Child_2 = Child_2

            else:
                Mutated_Child_2 = Child_2
        '''
        print()
        print("Mutated_Child_1:",Mutated_Child_1)
        print("Mutated_Child_2:",Mutated_Child_2)
        '''

        # For mutated child_1

        X0_num_Sum_MC_1 = 0
        Y0_num_Sum_MC_1 = 0

        z = 0
        t = 1


        for i in range(len(XY0)//2):
            X0_num_M1 = Mutated_Child_1[-t]*2**z
            X0_num_Sum_MC_1 += X0_num_M1
            t = t+1
            z = z+1

        p = 0
        u = 1+(len(XY0)//2)

        for i in range(len(XY0)//2):
            Y0_num_M1 = Mutated_Child_1[-u]*2**p
            Y0_num_Sum_MC_1 += Y0_num_M1
            u = u+1
            p = p+1


        Decoded_X_M1 = (X0_num_Sum_MC_1*Precision_X)+a_X
        Decoded_Y_M1 = (Y0_num_Sum_MC_1*Precision_Y)+a_Y
        '''
        print()
        print("Decoded_X_M1:",Decoded_X_M1)
        print("Decoded_Y_M1:",Decoded_Y_M1)
        '''
        P_1 = 0

        kf = KFold(n_splits=kfold)

        for train_index,test_index in kf.split(X):
            X_train,X_test = X.iloc[train_index],X.iloc[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]

            model1 = svm.SVC(kernel="rbf",C=Decoded_X_M1,gamma=Decoded_Y_M1)
            model1.fit(X_train,Y_train.ravel())
            PL1 = model1.predict(X_test)

            AC1 = model1.score(X_test,Y_test)

            OF_So_Far_1 = 1-(AC1)

            P_1 += OF_So_Far_1

        OF_So_Far_M1 = P_1/kfold


        # For mutated child_2

        X0_num_Sum_MC_2 = 0
        Y0_num_Sum_MC_2 = 0

        z = 0
        t = 1


        for i in range(len(XY0)//2):
            X0_num_M2 = Mutated_Child_2[-t]*2**z
            X0_num_Sum_MC_2 += X0_num_M2
            t = t+1
            z = z+1

        p = 0
        u = 1+(len(XY0)//2)

        for i in range(len(XY0)//2):
            Y0_num_M2 = Mutated_Child_2[-u]*2**p
            Y0_num_Sum_MC_2 += Y0_num_M2
            u = u+1
            p = p+1


        Decoded_X_M2 = (X0_num_Sum_MC_2*Precision_X)+a_X
        Decoded_Y_M2 = (Y0_num_Sum_MC_2*Precision_Y)+a_Y
        '''
        print()
        print("Decoded_X_M2:",Decoded_X_M2)
        print("Decoded_Y_M2:",Decoded_Y_M2)
        '''
        P_1 = 0

        kf = KFold(n_splits=kfold)

        for train_index,test_index in kf.split(X):
            X_train,X_test = X.iloc[train_index],X.iloc[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]

            model1 = svm.SVC(kernel="rbf",C=Decoded_X_M2,gamma=Decoded_Y_M2)
            model1.fit(X_train,Y_train.ravel())
            PL1 = model1.predict(X_test)

            AC1 = model1.score(X_test,Y_test)

            OF_So_Far_2 = 1-(AC1)

            P_1 += OF_So_Far_2

        OF_So_Far_M2 = P_1/kfold


        print()
        print("FV for Mutated Child #1 at Gen",Generation,":",OF_So_Far_M1)
        print("FV for Mutated Child #2 at Gen",Generation,":",OF_So_Far_M2)


        All_in_Generation_X_1_1_temp = Mutated_Child_1[np.newaxis]

        All_in_Generation_X_1_1 = np.column_stack((OF_So_Far_M1,All_in_Generation_X_1_1_temp))

        All_in_Generation_X_2_2_temp = Mutated_Child_2[np.newaxis]

        All_in_Generation_X_2_2 = np.column_stack((OF_So_Far_M2,All_in_Generation_X_2_2_temp))


        All_in_Generation_X_1 = np.vstack((All_in_Generation_X_1,All_in_Generation_X_1_1))
        All_in_Generation_X_2 = np.vstack((All_in_Generation_X_2,All_in_Generation_X_2_2))


        Save_Best_in_Generation_X = np.vstack((All_in_Generation_X_1,All_in_Generation_X_2))


        New_Population = np.vstack((New_Population,Mutated_Child_1,Mutated_Child_2))

        R_1 = []
        t = 0
        for i in All_in_Generation_X_1:

            if(All_in_Generation_X_1[t,:1]) <= min(All_in_Generation_X_1[:,:1]):
                R_1 = All_in_Generation_X_1[t,:]
            t = t+1


        Min_in_Generation_X_1 = R_1[np.newaxis]


        R_2 = []
        t = 0
        for i in All_in_Generation_X_2:

            if(All_in_Generation_X_2[t,:1]) <= min(All_in_Generation_X_2[:,:1]):
                R_2 = All_in_Generation_X_2[t,:]
            t = t+1


        Min_in_Generation_X_2 = R_2[np.newaxis]


        Family = Family+1


    t = 0
    R_11 = []
    for i in Save_Best_in_Generation_X:

        if (Save_Best_in_Generation_X[t,:1]) <= min(Save_Best_in_Generation_X[:,:1]):
            R_11 = Save_Best_in_Generation_X[t,:]
        t = t+1

    Final_Best_in_Generation_X = R_11[np.newaxis]


    t = 0
    R_22 = []
    for i in Save_Best_in_Generation_X:

        if (Save_Best_in_Generation_X[t,:1]) >= max(Save_Best_in_Generation_X[:,:1]):
            R_22 = Save_Best_in_Generation_X[t,:1]
        t = t+1

    Worst_Best_in_Generation_X = R_22[np.newaxis]


    Darwin_Guy = Final_Best_in_Generation_X[:]
    Not_So_Darwin_Guy = Worst_Best_in_Generation_X[:]


    Darwin_Guy = Darwin_Guy[0:,1:].tolist()
    Not_So_Darwin_Guy = Not_So_Darwin_Guy[0:,1:].tolist()

    Best_1 = np.where((New_Population==Darwin_Guy))
    Worst_1 = np.where((New_Population==Not_So_Darwin_Guy))


    New_Population[Worst_1] = Darwin_Guy

    n_list = New_Population


    Min_for_all_Generations_for_Mut_1 = np.vstack((Min_for_all_Generations_for_Mut_1,
                                                   Min_in_Generation_X_1))
    Min_for_all_Generations_for_Mut_2 = np.vstack((Min_for_all_Generations_for_Mut_2,
                                                   Min_in_Generation_X_2))


    Min_for_all_Generations_for_Mut_1_1 = np.insert(Min_in_Generation_X_1,0,Generation)
    Min_for_all_Generations_for_Mut_2_2 = np.insert(Min_in_Generation_X_2,0,Generation)


    Min_for_all_Generations_for_Mut_1_1_1 = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                                                       Min_for_all_Generations_for_Mut_1_1))

    Min_for_all_Generations_for_Mut_2_2_2 = np.vstack((Min_for_all_Generations_for_Mut_2_2_2,
                                                       Min_for_all_Generations_for_Mut_2_2))


    Generation = Generation+1



One_Final_Guy = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                           Min_for_all_Generations_for_Mut_2_2_2))


t = 0
Final_Here = []

for i in One_Final_Guy:
    if(One_Final_Guy[t,1]) <= min(One_Final_Guy[:,1]):
        Final_Here = One_Final_Guy[t,:]
    t = t+1

One_Final_Guy_Final = Final_Here[np.newaxis]

print()
print("Min in all Generations:",One_Final_Guy_Final)

print()
print("Final Solution:",One_Final_Guy_Final[:,2:])
print("Highest Accuracy:",(1-One_Final_Guy_Final[:,1]))

XY0_Encoded_After = One_Final_Guy_Final[:,2:][0]


z = 0
t = 1
X0_num_Sum_Encoded_After = 0

for i in range(len(XY0)//2):
    X0_num_Encoded_After = XY0_Encoded_After[-t]*2**z
    X0_num_Sum_Encoded_After += X0_num_Encoded_After
    t = t+1
    z = z+1

p = 0
u = 1+(len(XY0)//2)
Y0_num_Sum_Encoded_After = 0

for i in range(len(XY0)//2):
    Y0_num_Encoded_After = XY0_Encoded_After[-u]*2**p
    Y0_num_Sum_Encoded_After += Y0_num_Encoded_After
    u = u+1
    p = p+1


Decoded_X_Encoded_After = (X0_num_Sum_Encoded_After*Precision_X)+a_X
Decoded_Y_Encoded_After = (Y0_num_Sum_Encoded_After*Precision_Y)+a_Y

print()
print("Decoded_X_Encoded_After:",Decoded_X_Encoded_After)
print("Decoded_Y_Encoded_After:",Decoded_Y_Encoded_After)
print("--- %s seconds ---" % (time.time() - start_time))



Decoded_X: 856.6377758110294
Decoded_Y: 0.1482830286568804

GENERATION: #: 1

Family: #: 1

FV for Mutated Child #1 at Gen 1 : 0.24092409240924093
FV for Mutated Child #2 at Gen 1 : 0.18481848184818483

Family: #: 2

FV for Mutated Child #1 at Gen 1 : 0.1914191419141914
FV for Mutated Child #2 at Gen 1 : 0.19801980198019806

Family: #: 3

FV for Mutated Child #1 at Gen 1 : 0.23762376237623759
FV for Mutated Child #2 at Gen 1 : 0.18481848184818483

Family: #: 4

FV for Mutated Child #1 at Gen 1 : 0.17821782178217824
FV for Mutated Child #2 at Gen 1 : 0.22772277227722773

Family: #: 5

FV for Mutated Child #1 at Gen 1 : 0.22442244224422445
FV for Mutated Child #2 at Gen 1 : 0.1914191419141914

Family: #: 6

FV for Mutated Child #1 at Gen 1 : 0.22772277227722773
FV for Mutated Child #2 at Gen 1 : 0.20132013201320134

Family: #: 7

FV for Mutated Child #1 at Gen 1 : 0.22112211221122113
FV for Mutated Child #2 at Gen 1 : 0.17491749174917492

Family: #: 8

FV for Mutated Child #1 at Gen 1 :

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 2 : 0.2343234323432343
FV for Mutated Child #2 at Gen 2 : 0.2277227722772277

Family: #: 3

FV for Mutated Child #1 at Gen 2 : 0.17821782178217824
FV for Mutated Child #2 at Gen 2 : 0.22772277227722773

Family: #: 4

FV for Mutated Child #1 at Gen 2 : 0.18151815181518152
FV for Mutated Child #2 at Gen 2 : 0.21452145214521454

Family: #: 5

FV for Mutated Child #1 at Gen 2 : 0.23102310231023104
FV for Mutated Child #2 at Gen 2 : 0.2244224422442244

Family: #: 6

FV for Mutated Child #1 at Gen 2 : 0.2343234323432343
FV for Mutated Child #2 at Gen 2 : 0.22442244224422445

Family: #: 7

FV for Mutated Child #1 at Gen 2 : 0.2376237623762376
FV for Mutated Child #2 at Gen 2 : 0.1914191419141914

Family: #: 8

FV for Mutated Child #1 at Gen 2 : 0.22442244224422445
FV for Mutated Child #2 at Gen 2 : 0.23762376237623759

Family: #: 9

FV for Mutated Child #1 at Gen 2 : 0.18481848184818483
FV for Mutated Child #2 at Gen 2 : 0.19141914191419143

Family: #: 10

FV f

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 3 : 0.18151815181518152
FV for Mutated Child #2 at Gen 3 : 0.1881188118811881

Family: #: 3

FV for Mutated Child #1 at Gen 3 : 0.2376237623762376
FV for Mutated Child #2 at Gen 3 : 0.19801980198019806

Family: #: 4

FV for Mutated Child #1 at Gen 3 : 0.1848184818481848
FV for Mutated Child #2 at Gen 3 : 0.19471947194719474

Family: #: 5

FV for Mutated Child #1 at Gen 3 : 0.22112211221122113
FV for Mutated Child #2 at Gen 3 : 0.22442244224422445

Family: #: 6

FV for Mutated Child #1 at Gen 3 : 0.18151815181518152
FV for Mutated Child #2 at Gen 3 : 0.2343234323432343

Family: #: 7

FV for Mutated Child #1 at Gen 3 : 0.22442244224422445
FV for Mutated Child #2 at Gen 3 : 0.2343234323432343

Family: #: 8

FV for Mutated Child #1 at Gen 3 : 0.1881188118811881
FV for Mutated Child #2 at Gen 3 : 0.20132013201320134

Family: #: 9

FV for Mutated Child #1 at Gen 3 : 0.18151815181518152
FV for Mutated Child #2 at Gen 3 : 0.2442244224422442

Family: #: 10

FV fo

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 49 : 0.17491749174917492
FV for Mutated Child #2 at Gen 49 : 0.18151815181518152

Family: #: 2

FV for Mutated Child #1 at Gen 49 : 0.17821782178217824
FV for Mutated Child #2 at Gen 49 : 0.17491749174917492

Family: #: 3

FV for Mutated Child #1 at Gen 49 : 0.17491749174917492
FV for Mutated Child #2 at Gen 49 : 0.1716171617161716

Family: #: 4

FV for Mutated Child #1 at Gen 49 : 0.1716171617161716
FV for Mutated Child #2 at Gen 49 : 0.18151815181518152

Family: #: 5

FV for Mutated Child #1 at Gen 49 : 0.17491749174917492
FV for Mutated Child #2 at Gen 49 : 0.17821782178217824

Family: #: 6

FV for Mutated Child #1 at Gen 49 : 0.18151815181518152
FV for Mutated Child #2 at Gen 49 : 0.17821782178217824

Family: #: 7

FV for Mutated Child #1 at Gen 49 : 0.18151815181518152
FV for Mutated Child #2 at Gen 49 : 0.17821782178217824

Family: #: 8

FV for Mutated Child #1 at Gen 49 : 0.1716171617161716
FV for Mutated Child #2 at Gen 49 : 0.1716171617161716

F

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.



FV for Mutated Child #1 at Gen 50 : 0.17821782178217824
FV for Mutated Child #2 at Gen 50 : 0.17161716171617167

Family: #: 2

FV for Mutated Child #1 at Gen 50 : 0.16831683168316833
FV for Mutated Child #2 at Gen 50 : 0.17821782178217824

Family: #: 3

FV for Mutated Child #1 at Gen 50 : 0.1716171617161716
FV for Mutated Child #2 at Gen 50 : 0.16831683168316833

Family: #: 4

FV for Mutated Child #1 at Gen 50 : 0.1716171617161716
FV for Mutated Child #2 at Gen 50 : 0.17821782178217827

Family: #: 5

FV for Mutated Child #1 at Gen 50 : 0.17821782178217824
FV for Mutated Child #2 at Gen 50 : 0.1716171617161716

Family: #: 6

FV for Mutated Child #1 at Gen 50 : 0.17821782178217824
FV for Mutated Child #2 at Gen 50 : 0.18151815181518152

Family: #: 7

FV for Mutated Child #1 at Gen 50 : 0.1716171617161716
FV for Mutated Child #2 at Gen 50 : 0.18151815181518152

Family: #: 8

FV for Mutated Child #1 at Gen 50 : 0.1716171617161716
FV for Mutated Child #2 at Gen 50 : 0.17821782178217824

Fa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:615: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


# Accuracy on selected features after hypertuning

In [ ]:
start_time = time.time()
kfold = 10

SVMClass = svm.SVC()
Aa1 = 0

kf = KFold(n_splits=kfold)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    model1 = SVMClass
    model1.fit(X_train, Y_train.ravel())
    Pa_1=model1.predict(X_test)
    AC1=model1.score(X_test,Y_test)
    
    Aa1 += AC1
       
print()
print("Accuracy for SVM without GA: %f" % (Aa1/kfold))



SVMClass = svm.SVC(kernel='rbf', C=27.16, gamma=0.096)
Aa1 = 0

kf = KFold(n_splits=kfold)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    model1 = SVMClass
    model1.fit(X_train, Y_train.ravel())
    Pa_1=model1.predict(X_test)
    AC1=model1.score(X_test,Y_test)
    
    Aa1 += AC1
       
print()
print("Accuracy for SVM with GA: %f" % (Aa1/kfold))
print("--- %s seconds ---" % (time.time() - start_time))


Accuracy for SVM without GA: 0.834839

Accuracy for SVM with GA: 0.838387
--- 0.09412479400634766 seconds ---


# Inferences

In [ ]:
from IPython.display import display 
 
dict = {'Metric' : ['Without GA','Hyperparameter tuning','After hyp.tuning (whole dataset)','Feature selection','Hyperparameter tuning(selected features)',
                    'After FS + hyp. tuning'], 

        'Accuracy' : [82.17,84.81,83.15,84.8,84.15,83.8],
        'Hyperparameters' : ['-', 'C-11,G-0.05', '-', '-', 'C-27.16,G-0.096','-'],
        'Time'            : ['0.03829455375671387s', '182.94120407104492s', '0.0665895938873291s', '140.32491946220398s', '270.0007164478302', '0.09412479400634766s']}
df = pd.DataFrame(dict)  
display(df)

,Metric,Accuracy,Hyperparameters,Time
0,Without GA,82.17,-,0.03829455375671387s
1,Hyperparameter tuning,84.81,"C-11,G-0.05",182.94120407104492s
2,After hyp.tuning (whole dataset),83.15,-,0.0665895938873291s
3,Feature selection,84.80,-,140.32491946220398s
4,Hyperparameter tuning(selected features),84.15,"C-27.16,G-0.096",270.0007164478302
5,After FS + hyp. tuning,83.80,-,0.09412479400634766s
